In [1]:
proj_list = [
    'boringssl', 'c-ares',
    'freetype2', 'guetzli',
    'harfbuzz', 'lcms',
    'libpng', 'libssh',
    'libxml2', 'pcre2',
    'proj4', 're2',
    'sqlite3', 'vorbis',
    'woff2', 'wpantund'
]

In [2]:
target_project = 0

In [3]:
from sklearn.model_selection import train_test_split
import torch
import torch, gc

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import timeit

import data
import data_loader as dl
import initializer as init
import trainer
import tester
# import predictor
import model_util as mu

import os

In [4]:
gc.collect()
torch.cuda.empty_cache()

print(torch.cuda.memory_summary(device=None, abbreviated=False))

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [5]:
prefix_np, postfix_np, label_np, label_len_np = data.getTrainData(proj_list, proj_list[target_project])

Getting data for "boringssl" from "c-ares"
Getting data for "boringssl" from "freetype2"
Getting data for "boringssl" from "guetzli"
Getting data for "boringssl" from "harfbuzz"
Getting data for "boringssl" from "lcms"
Getting data for "boringssl" from "libpng"
Getting data for "boringssl" from "libssh"
Getting data for "boringssl" from "libxml2"
Getting data for "boringssl" from "pcre2"
Getting data for "boringssl" from "proj4"
Getting data for "boringssl" from "re2"
Getting data for "boringssl" from "sqlite3"
Getting data for "boringssl" from "vorbis"
Getting data for "boringssl" from "woff2"
Getting data for "boringssl" from "wpantund"


In [6]:
test_prefix, test_postfix, test_label, test_label_len = data.getTestData(proj_list[target_project])

In [7]:
train_prefix, val_prefix, train_postfix, val_postfix, train_label, val_label = train_test_split(
    prefix_np, postfix_np, label_np, test_size = 0.2, random_state = 43
)

In [8]:
train_dataloader, val_dataloader, test_dataloader =\
    dl.data_loader(
        train_prefix, train_postfix,
        val_prefix, val_postfix,
        test_prefix, test_postfix,
        train_label, val_label, test_label,
        batch_size=1000
    )

In [9]:
overall_title = 'developing_1'

In [10]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('../tensorboard/'+overall_title+'/tests')

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [12]:
# ====================
# set parameters here
# ====================

title = proj_list[target_project] + '_' + overall_title + '_wd'
epochs = 40

max_len, source_code_tokens, token_choices = data.getInfo()

learning_rate = 0.001
weight_decay = 1e-4 

embed_dim = 128
hidden_size = 200
n_layers = 2
output_size = max(token_choices) + 2
dropout = 0.0
max_length = max_len
input_size = max(token_choices) + 2
device = device

model_name = "seq2seq"
optim_name = "Adam"
loss_fn_name = "CEL"

teacher_forcing_ratio = 100.0

In [13]:
trainer.set_seed(42)

loss_fn, prefix_pack, postfix_pack, attn_pack = init.initialize_model(
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    n_layers=n_layers,
    output_size=output_size,
    dropout=dropout,
    max_length=max_length,
    input_size=input_size,
    device=device
)

In [14]:
start_time = timeit.default_timer()

trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    loss_fn=loss_fn,
    prefix_pack=prefix_pack,
    postfix_pack=postfix_pack,
    attn_pack=attn_pack,
    teacher_forcing_ratio=teacher_forcing_ratio
)

end_time = (timeit.default_timer() - start_time) / 60.0

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   6.708803   | 95.086936  | 10.711761 | 90.59  | 319.03
   2    |   5.847102   | 97.141494  | 9.998756 | 92.23  | 315.36
   3    |   5.732020   | 97.503449  | 9.768381 | 92.52  | 309.39
   4    |   5.677894   | 97.676615  | 9.649593 | 92.92  | 305.66
   5    |   5.642586   | 97.789800  | 9.449209 | 93.19  | 316.07
   6    |   5.614899   | 97.881977  | 9.418554 | 93.40  | 306.90
   7    |   5.591903   | 97.954862  | 9.409832 | 93.53  | 311.42
   8    |   5.572501   | 98.020753  | 9.335182 | 93.52  | 324.00
   9    |   5.554717   | 98.081002  | 9.243919 | 93.86  | 333.38
  10    |   5.538787   | 98.134889  | 9.132729 | 93.93  | 319.35
  11    |   5.524139   | 98.184779  | 9.139077 | 94.06  | 325.90
  12    |   5.510841   | 98.235303  | 9.141760 | 94.01  | 322.36
  13    |   5.497765   | 98.277407  | 9.168933 | 94.

In [15]:
mu.saveModel(overall_title, title, prefix_pack, postfix_pack, attn_pack)

In [16]:
# model = mu.getModel(title)
# print(model)

In [17]:
loss, acc = tester.test(
    test_dataloader=test_dataloader,
    device=device,
    loss_fn=loss_fn,
    prefix_pack=prefix_pack,
    postfix_pack=postfix_pack,
    attn_pack=attn_pack
)

test loss:  11.661640846875812
test acc:  91.45104350902017


In [18]:
with open('../stat/'+overall_title, 'a') as f:
        text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(end_time, 3)) + ' min\n'
        f.write(text)

In [19]:
# mu.graphModel(train_dataloader, model, writer)